In [1]:
import pandas as pd
import dash
import dash_table
from dash.dependencies import Input, Output, State, ALL
from dash import dcc, html
import random
import string
from datetime import datetime

# Load data
path = r"C:\Users\HomePC\Documents\HTMLcSS\Nutrition\Data\ingredient.csv"
dfx = pd.read_csv(path)
dfx.drop(columns="PRICE/KG", inplace=True)

# Define columns, with Quantity right after INGREDIENT
columns = [
    'INGREDIENT', 'QUANTITY', 'PRICE/KG', 'QUANTITY PRICE', 'DRY MATTER', 'CP', 'FAT', 'FIBRE', 'CAL.', 'PHOS.TOTAL',
    'AVAIL PHOS', 'ME/POULT', 'ME/SWINE', 'METH', 'CYSTINE', 'METH+CYST',
    'LYSINE', 'TRYPTOPHAN', 'THREONINE', 'VIT A IU/GM', 'VIT D3 IU/GM',
    'VIT E IU/GM', 'RIBOFLAVIN', 'PANTO ACID', 'CHOLINE', 'B 12', 'NIACIN',
    'XANTHOPYL', 'SALT', 'SODIUM', 'POTASSIUM', 'MAGNESIUM', 'SULPHUR',
    'MANGANESE', 'IRON', 'COPPER', 'ZINC', 'SELENIUM', 'IODINE', 'LINOLEIC A'
]

# Create a DataFrame for the main table
df = pd.DataFrame(columns=columns)

# Default data for the vitamins/minerals table
default_data = {
    "vitamin/mineral": [
        "Vitamin A IU/GM", "Vitamin D3 IU/GM", "Vitamin E IU/GM", "Riboflavin",
        "Pantothenic acid", "Choline", "Vitamin B 12", "Niacin",
        "Xantophyl", "Linoleic acid", "Salt", "Sodium", "Potassium",
        "Magnesium", "Sulphur", "Manganese", "Iron", "Copper", "Zinc",
        "Selenium", "Iodine"
    ],
    "PRICE/KG": [0] * 21,  
    "QUANTITY": [0] * 21,  
    "AMOUNT": [0] * 21,    
}

# Convert default_data to DataFrame for easy manipulation
df2 = pd.DataFrame(default_data)

# Create options for the dropdown from the vitamins list
dropdown_options = [{'label': vit, 'value': vit} for vit in default_data['vitamin/mineral']]

# Initialize Dash app
app = dash.Dash(__name__, prevent_initial_callbacks=True, meta_tags=[{"name": "viewport", "content": "width=device-width"}])

app.layout = html.Div(children=[
    html.Div(children=[
        html.Div(id="sidebar", children=[
            dcc.Dropdown(
                id='ingredient_dd',
                options=[{'label': ingredient, 'value': ingredient} for ingredient in dfx['INGREDIENT'].unique()],
                multi=True, placeholder="Select ingredients..."
            ),

            html.Div(children=[
                dcc.Dropdown(
                    id='vit_dd',
                    options=dropdown_options,
                    multi=True, placeholder="Select vitamins/minerals...")
            ], style={"position": "relative", "top": "50px"})

        ], className="sidebox"),

        html.Div(children=[
            html.Div(id="topbar"),
            html.Div(children=[
                html.Div(
                    id="firstbar", children=[
                        dash_table.DataTable(
                            id='ingredient_table',
                            columns=[{'name': col, 'id': col, 'editable': col in ['QUANTITY', 'PRICE/KG']} for col in columns],
                            data=[],  # Initially empty
                            style_table={
                                'overflowY': 'auto',  # Enable vertical scroll
                                'overflowX': 'auto',  # Enable horizontal scroll
                                'height': '100%',     # Ensure the table fills the container height
                                'width': '100%',      # Ensure the table fills the container width
                            },
                            style_cell={
                                'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
                                'whiteSpace': 'normal',
                                'textAlign': 'left',
                                'padding': '5px'
                            },
                            style_data={
                                'height': 'auto',  # Allow rows to adjust height but limit it below
                                'maxHeight': '65px',  # Set a max height for each row
                                'overflow': 'hidden',  # Hide overflow to prevent table expansion
                            },
                            page_current=0,  # Start on the first page
                            page_size=6,     # Six rows per page
                            page_action="native",  # Native pagination mode
                            style_header={
                                'backgroundColor': 'lightgrey',
                                'fontWeight': 'bold'
                            }
                        ),

                        html.Div(children=[
                            dash_table.DataTable(
                                id='minerals-vitamins-table',
                                columns=[
                                    {"name": "INGREDIENT", "id": "INGREDIENT"},
                                    {"name": "QUANTITY", "id": "QUANTITY"},
                                    {"name": "PRICE/KG", "id": "PRICE/KG"},
                                    {"name": "AMOUNT", "id": "AMOUNT"}
                                ],
                                data=[],  
                                editable=True,
                                row_deletable=True,
                                style_table={'height': '200px', 'overflowY': 'auto', 'overflowX': 'auto'},
                                style_cell={'textAlign': 'left', 'fontFamily': 'Arial', 'fontSize': 15},
                                style_header={'backgroundColor': 'lightgrey', 'color': 'black', 'fontWeight': 'bold'})
                        ], style={"position": "relative", "top": "145px"})
                    ]),

                html.Div(
                    id="secondbar",
                    style={
                        'height': '475px',
                        'overflowY': 'auto',
                        'padding': '10px',
                        'border': '1px solid #ddd',
                        'boxSizing': 'border-box',
                    },
                    children=[
                        # Date and Time
                        html.Div(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", style={'fontWeight': 'bold', 'marginBottom': '10px'}),

                        # Feed Name Input
                        html.Div([
                            html.Label("Feed Name:", style={'fontWeight': 'bold'}),
                            dcc.Input(id='feed_name', type='text', placeholder='Enter feed name', style={'marginLeft': '10px'})
                        ], style={'marginBottom': '10px'}),

                        # Feed Code Display
                        html.Div([
                            html.Label("Feed Code:", style={'fontWeight': 'bold'}),
                            html.Span(id='feed_code', style={'marginLeft': '10px'})
                        ], style={'marginBottom': '10px'}),

                        # Feed Analysis Report Heading
                        html.H2("Feed Analysis Report", style={'fontWeight': 'bold'}),

                        # Table for selected ingredients
                        dash_table.DataTable(
                            id='report_table',
                            columns=[{'name': col, 'id': col} for col in ['INGREDIENT', 'PRICE/KG', 'QUANTITY', 'AMOUNT']],
                            data=[],  # This will be filled dynamically
                            style_table={
                                'width': '100%',
                            },
                            style_cell={
                                'textAlign': 'left',
                                'padding': '5px'
                            },
                            style_header={
                                'backgroundColor': 'lightgrey',
                                'fontWeight': 'bold'
                            }
                        ),

                        # Table for nutrient composition
                        dash_table.DataTable(
                            id='nutrient_table',
                            columns=[{'name': 'Nutrient Composition', 'id': 'Nutrient'},
                                     {'name': 'Actual', 'id': 'Actual'}],
                            data=[],  # This will be filled dynamically
                            style_table={
                                'width': '100%',
                            },
                            style_cell={
                                'textAlign': 'left',
                                'padding': '5px'
                            },
                            style_header={
                                'backgroundColor': 'lightgrey',
                                'fontWeight': 'bold'
                            }
                        ),

                        # Print Button
                        html.Button("Print", id='print_button', n_clicks=0, style={'marginTop': '20px', 'fontWeight': 'bold'})
                    ]
                )
            ], className="rectangle")
        ], className="box_arrange")
    ], className="all_box")
], className="container")


@app.callback(
    Output('feed_code', 'children'),
    [Input('feed_name', 'value')]
)
def generate_feed_code(feed_name):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=4))


@app.callback(
    Output('print_button', 'style'),
    [Input('print_button', 'n_clicks')]
)
def hide_print_button(n_clicks):
    if n_clicks > 0:
        return {'display': 'none'}
    return {}

@app.callback(
    [Output('ingredient_table', 'data'),
     Output('report_table', 'data'),
     Output('nutrient_table', 'data'),
     Output('minerals-vitamins-table', 'data')],
    [Input('ingredient_dd', 'value'),
     Input('ingredient_table', 'data_timestamp'),
     Input('vit_dd', "value")],
    [State('ingredient_table', 'data'),
     State('ingredient_table', 'data')]
)
def update_table(selected_ingredients, _, ingre_current_data, selected_vitamins, vit_current_data):
    print("Selected Ingredients:", selected_ingredients)
    print("Current Ingredient Table Data:", ingre_current_data)
    print("Selected Vitamins:", selected_vitamins)
    print("Current Vitamins Table Data:", vit_current_data)
    
    if not selected_ingredients:
        selected_ingredients = []

    if not selected_vitamins:
        selected_vitamins = []

    # Process ingredient selection
    updated_ingredients = []
    for ingredient in selected_ingredients:
        matching_row = dfx[dfx['INGREDIENT'] == ingredient].iloc[0].to_dict()
        updated_ingredients.append(matching_row)

    df_update = pd.DataFrame(updated_ingredients)

    # Ensure correct columns are present
    df_update['QUANTITY'] = df_update.get('QUANTITY', 0)
    df_update['PRICE/KG'] = df_update.get('PRICE/KG', 0)
    df_update['QUANTITY PRICE'] = df_update['QUANTITY'] * df_update['PRICE/KG']

    # Vitamins/minerals handling
    selected_vitamins_df = df2[df2['vitamin/mineral'].isin(selected_vitamins)]
    vitamins_minerals_table_data = selected_vitamins_df.to_dict('records')

    # Update report table
    report_table_data = df_update[['INGREDIENT', 'PRICE/KG', 'QUANTITY', 'QUANTITY PRICE']].to_dict('records')

    # Handle Total row
    total_row = {
        'INGREDIENT': 'Total',
        'PRICE/KG': '',
        'QUANTITY': '',
        'QUANTITY PRICE': df_update['QUANTITY PRICE'].sum()
    }
    report_table_data.append(total_row)

    # Nutrient composition update
    nutrient_table_data = []
    for nutrient in columns[4:]:
        nutrient_sum = df_update[nutrient].sum()
        nutrient_table_data.append({'Nutrient': nutrient, 'Actual': nutrient_sum})

    return df_update.to_dict('records'), report_table_data, nutrient_table_data, vitamins_minerals_table_data


if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\HomePC\AppData\Local\Temp\ipykernel_1036\2558431299.py:3: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [ ]:
import pandas as pd
import dash
import dash_table
from dash.dependencies import Input, Output, State
from dash import dcc, html
import random
import string
from datetime import datetime

# Load data
path = r"C:\Users\HomePC\Documents\HTMLcSS\Nutrition\Data\ingredient.csv"
dfx = pd.read_csv(path)
dfx.drop(columns="PRICE/KG", inplace=True)

# Define columns, with Quantity right after INGREDIENT
columns = [
    'INGREDIENT', 'QUANTITY', 'PRICE/KG', 'QUANTITY PRICE', 'DRY MATTER', 'CP', 'FAT', 'FIBRE', 'CAL.', 'PHOS.TOTAL',
    'AVAIL PHOS', 'ME/POULT', 'ME/SWINE', 'METH', 'CYSTINE', 'METH+CYST',
    'LYSINE', 'TRYPTOPHAN', 'THREONINE', 'VIT A IU/GM', 'VIT D3 IU/GM',
    'VIT E IU/GM', 'RIBOFLAVIN', 'PANTO ACID', 'CHOLINE', 'B 12', 'NIACIN',
    'XANTHOPYL', 'SALT', 'SODIUM', 'POTASSIUM', 'MAGNESIUM', 'SULPHUR',
    'MANGANESE', 'IRON', 'COPPER', 'ZINC', 'SELENIUM', 'IODINE', 'LINOLEIC A'
]

df = pd.DataFrame(columns=columns)


default_data = {
    "vitamin/mineral": [
        "Vitamin A IU/GM", "Vitamin D3 IU/GM", "Vitamin E IU/GM", "Riboflavin",
        "Pantothenic acid", "Choline", "Vitamin B 12", "Niacin",
        "Xantophyl", "Linoleic acid", "Salt", "Sodium", "Potassium",
        "Magnesium", "Sulphur", "Manganese", "Iron", "Copper", "Zinc",
        "Selenium", "Iodine"
    ],
    "PRICE/KG": [0] * 21,  
    "QUANTITY": [0] * 21,  
    "AMOUNT": [0] * 21,    
}

# Convert default_data to DataFrame for easy manipulation
df2 = pd.DataFrame(default_data)

# Create options for the dropdown from the vitamins list
dropdown_options = [{'label': vit, 'value': vit} for vit in default_data['vitamin/mineral']]

# Initialize Dash app
app = dash.Dash(__name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}])

app.layout = html.Div(children=[
    html.Div(children=[
        html.Div(id="sidebar", children=[
            dcc.Dropdown(
                id='ingredient_dd',
                options=[{'label': ingredient, 'value': ingredient} for ingredient in dfx['INGREDIENT'].unique()],
                multi=True, placeholder="Select ingredients..."
            ),

            html.Div(children = [
                dcc.Dropdown(
                id='vit_dd',
                options= dropdown_options,
                multi=True, placeholder="Select vitamins/minerals...")

        ], style = {"position": "relative", "top": "50px"})

        ], className="sidebox"),

        html.Div(children=[
            html.Div(id="topbar"),
            html.Div(children=[
                html.Div(
                    id="firstbar", children=[
                        dash_table.DataTable(
                            id='ingredient_table',
                            columns=[{'name': col, 'id': col, 'editable': col in ['QUANTITY', 'PRICE/KG']} for col in columns],
                            data=[],  # Initially empty
                            style_table={
                                'overflowY': 'auto',  # Enable vertical scroll
                                'overflowX': 'auto',  # Enable horizontal scroll
                                'height': '100%',     # Ensure the table fills the container height
                                'width': '100%',      # Ensure the table fills the container width
                            },
                            style_cell={
                                'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
                                'whiteSpace': 'normal',
                                'textAlign': 'left',
                                'padding': '5px'
                            },
                            style_data={
                                'height': 'auto',  # Allow rows to adjust height but limit it below
                                'maxHeight': '65px',  # Set a max height for each row
                                'overflow': 'hidden',  # Hide overflow to prevent table expansion
                            },
                            page_current=0,  # Start on the first page
                            page_size=6,     # Six rows per page
                            page_action="native",  # Native pagination mode
                            style_header={
                                'backgroundColor': 'lightgrey',
                                'fontWeight': 'bold'
                            }
                        ),

                        html.Div(children = [dash_table.DataTable(
                            id='minerals-vitamins-table',
                            columns=[
                            {"name": "Ingredients", "id": "Ingredients"},
                            {"name": "QUANTITY", "id": "QUANTITY"},
                            {"name": "PRICE/KG", "id": "PRICE/KG"},
                            {"name": "AMOUNT", "id": "AMOUNT"}
                        ],
                            data=[],  
                            editable=True,
                            row_deletable=True,
                            style_table={'height': '200px', 'overflowY': 'auto', 'overflowX': 'auto'},
                            style_cell={'textAlign': 'left', 'fontFamily': 'Arial', 'fontSize': 15},
                            style_header={'backgroundColor': 'lightgrey', 'color': 'black',
                                          'fontWeight': 'bold'})
                ], style = {"position": "relative", "top": "145px"})
                    
                    ]),

                html.Div(
                    id="secondbar",
                    style={
                        'height': '475px',
                        'overflowY': 'auto',
                        'padding': '10px',
                        'border': '1px solid #ddd',
                        'boxSizing': 'border-box',
                    },
                    children=[
                        # Date and Time
                        html.Div(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", style={'fontWeight': 'bold', 'marginBottom': '10px'}),

                        # Feed Name Input
                        html.Div([
                            html.Label("Feed Name:", style={'fontWeight': 'bold'}),
                            dcc.Input(id='feed_name', type='text', placeholder='Enter feed name', style={'marginLeft': '10px'})
                        ], style={'marginBottom': '10px'}),

                        # Feed Code Display
                        html.Div([
                            html.Label("Feed Code:", style={'fontWeight': 'bold'}),
                            html.Span(id='feed_code', style={'marginLeft': '10px'})
                        ], style={'marginBottom': '10px'}),

                        # Feed Analysis Report Heading
                        html.H2("Feed Analysis Report", style={'fontWeight': 'bold'}),

                        # Table for selected ingredients
                        dash_table.DataTable(
                            id='report_table',
                            columns=[{'name': col, 'id': col} for col in ['INGREDIENT', 'PRICE/KG', 'QUANTITY', 'AMOUNT']],
                            data=[],  # This will be filled dynamically
                            style_table={
                                'width': '100%',
                            },
                            style_cell={
                                'textAlign': 'left',
                                'padding': '5px'
                            },
                            style_header={
                                'backgroundColor': 'lightgrey',
                                'fontWeight': 'bold'
                            }
                        ),

                        # Table for nutrient composition
                        dash_table.DataTable(
                            id='nutrient_table',
                            columns=[{'name': 'Nutrient Composition', 'id': 'Nutrient'},
                                     {'name': 'Actual', 'id': 'Actual'}],
                            data=[],  # This will be filled dynamically
                            style_table={
                                'width': '100%',
                            },
                            style_cell={
                                'textAlign': 'left',
                                'padding': '5px'
                            },
                            style_header={
                                'backgroundColor': 'lightgrey',
                                'fontWeight': 'bold'
                            }
                        ),

                        # Print Button
                        html.Button("Print", id='print_button', n_clicks=0, style={'marginTop': '20px', 'fontWeight': 'bold'})
                    ]
                )
            ], className="rectangle")
        ], className="box_arrange")
    ], className="all_box")
], className="container")